In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
import jupyros as jr
import rospy
from tf.transformations import euler_from_quaternion
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from rt2_assignment1.srv import Command
from geometry_msgs.msg import Twist

#Control node initialization
rospy.init_node('robot_control_node')



## Control Block

### Start and stop buttons


In [2]:
# Start and stop button definition
start  = Button(description='Start',layout=Layout(width='auto', align="center", grid_area='Start'),style=ButtonStyle(button_color='green'))
stop  = Button(description='Stop',layout=Layout(width='auto', grid_area='Stop'),style=ButtonStyle(button_color='red'))
output = widgets.Output()

#Node initialization
ui_client = rospy.ServiceProxy('/user_interface', Command)
random_status = False

# Output of the buttons
def start_click(start):
    global random_status
    with output:
        ui_client("start")
        random_status = True
        print('Start clicked')
        

def stop_click(stop):
    global random_status
    with output:
        ui_client("stop")
        random_status = False
        print('Stop clicked')

# Callback definition
start.on_click(start_click)
stop.on_click(stop_click)

# Buttons display
buttons = GridBox(children=[start, stop],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='50% 50%',grid_template_areas='''"Start Stop "'''))
widgets.VBox([buttons, output])


### Slider commands

In [3]:
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

msg = Twist()

a = widgets.FloatSlider(min=-1.0, max=1.0, description='Linear vel.')
display(a)

b = widgets.FloatSlider(min=-1.0, max=1.0, description='Angular vel.')
display(b)


def on_value_change(change):
    global msg, random_status
    if random_status == False:
        msg.linear.x = change['new']
        pub.publish(msg)
    else:
        print("Command ignored, random position movement active")
    
def on_valueang_change(change):
    global msg, random_status
    if random_status == False:
        msg.angular.z = change['new']
        pub.publish(msg)
    else:
        print("Command ignored, random position movement active")

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')


FloatSlider(value=0.0, description='Linear vel.', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='Angular vel.', max=1.0, min=-1.0)

### Control via buttons

In [4]:
up  = Button(description='W',layout=Layout(width='auto', align="center", grid_area='W'),style=ButtonStyle(button_color='gray'))
left  = Button(description='A',layout=Layout(width='auto', grid_area='A'),style=ButtonStyle(button_color='gray'))
stop_asd  = Button(description='Alt',layout=Layout(width='auto', grid_area='Alt'),style=ButtonStyle(button_color='red'))
right = Button(description='D',layout=Layout(width='auto', grid_area='D'),style=ButtonStyle(button_color='gray'))
down = Button(description='S',layout=Layout(width='auto', grid_area='S'),style=ButtonStyle(button_color='gray'))

# Output of the buttons for moving
def up_click(up):
    global random_status
    if random_status == False:
        msg.linear.x = 0.5
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Go straight")
    else:
        print("Command ignored, random position movement active")       
        
def down_click(down):
    global random_status
    if random_status == False:
        msg.linear.x = -0.5
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Shrimp style")
    else:
        print("Command ignored, random position movement active")  
        
def right_click(right):
    global random_status
    if random_status == False:
        msg.linear.x = 0.5
        msg.angular.z = 1.0
        pub.publish(msg)
        print("Turn right")
    else:
        print("Command ignored, random position movement active")  
        
def left_click(left):
    global random_status
    if random_status == False:
        msg.linear.x = 0.5
        msg.angular.z = -1.0
        pub.publish(msg)
        print("Turn left")
    else:
        print("Command ignored, random position movement active")  

        
# Output of the button for stopping       
def stop_asd_click(stop_asd):
    global random_status
    if random_status == False:
        with output_buttons:
            msg.linear.x = 0.0
            msg.angular.z = 0.0
            pub.publish(msg)
        print('Robot stopped')
    else:
        print("Command ignored, random position movement active")  

# Callback definition
up.on_click(up_click)
down.on_click(down_click)
right.on_click(right_click)
left.on_click(left_click)
stop_asd.on_click(stop_asd_click)

# Buttons display
buttons = GridBox(children=[start, stop],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='50% 50%',grid_template_areas='''"Start Stop "'''))
widgets.VBox([buttons, output])

GridBox(children=[up,  left, stop_asd, right, down],layout=Layout(width='50%',grid_template_rows='auto auto',grid_template_columns='33% 33% 33%',grid_template_areas='''" . W . ""A Alt D "".S ."'''))

GridBox(children=(Button(description='W', layout=Layout(grid_area='W', width='auto'), style=ButtonStyle(button…

## Display

In [5]:
# First set up the figures and the axis 
    # Robot position and orientation
fig, ax = plt.subplots()
ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Robot positon")
line, = ax.plot([], [], 'ro')
x_data = []
y_data = []
theta_data = []

    # Velocity feedback
fig_vel, ax_vel = plt.subplots()
plt.title("Velocity check")
velocity_set = []
velocity_real = []
angular_set = []
angular_real = []
    
    
    
    # Boxplot of target reached
fig_bp, ax_bp = plt.subplots()
plt.title("Target  reached")
target_reached = []
target_cancelled = []

    # Histogram for time required for reaching a target
    
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def init():
    line.set_data([], [])
    return (line,)

def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    quaternion = (msg.pose.pose.orientation.x, msg.pose.pose.orientation.y,msg.pose.pose.orientation.z, msg.pose.pose.orientation.w)
    euler_angles = euler_from_quaternion(quaternion)
    theta_data.append(euler_angles[2])    
    
def animate(i):
    global arrow 
    line.set_data(x_data, y_data)
    #ax.patches.remove(arrow)
    
    arrow = plt.Arrow(x_data[-1], y_data[-1], dx=np.cos(theta_data[-1]), dy=np.sin(theta_data[-1]), width=0.1, color='red')
    ax.add_patch(arrow)
    return (line, arrow)

jr.subscribe('/odom', Odometry, odom_callback)

In [10]:
# call the animator. blit=True means only re-draw the parts that have changed.)

anim = animation.FuncAnimation(fig, animate, init_func=init,frames=100, interval=20, blit=True)